In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
# Read the dataset
training = spark.read.csv('test2.csv',header=True,inferSchema=True)
training.show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudanshu| 30|         8| 25000|
|   Sunny| 29|         6| 20000|
|   Shubh| 28|         4| 15000|
|  Harsha| 24|         1| 40000|
|  Mahesh| 26|         3| 35000|
|   Parul| 27|         5| 38000|
+--------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [7]:
# [Age,Experience] ---> new feature ----> independent feature 

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [10]:
output = featureassembler.transform(training)

In [11]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|Age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|   Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudanshu| 30|         8| 25000|          [30.0,8.0]|
|   Sunny| 29|         6| 20000|          [29.0,6.0]|
|   Shubh| 28|         4| 15000|          [28.0,4.0]|
|  Harsha| 24|         1| 40000|          [24.0,1.0]|
|  Mahesh| 26|         3| 35000|          [26.0,3.0]|
|   Parul| 27|         5| 38000|          [27.0,5.0]|
+--------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data = output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,6.0]| 20000|
|          [28.0,4.0]| 15000|
|          [24.0,1.0]| 40000|
|          [26.0,3.0]| 35000|
|          [27.0,5.0]| 38000|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [17]:
regressor.coefficients

DenseVector([-14066.6667, 9333.3333])

In [18]:
regressor.intercept

371933.3333335519

In [19]:
# Prediction
pred_result = regressor.evaluate(test_data)

In [20]:
pred_result.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,1.0]| 40000|43666.666666674544|
|         [31.0,10.0]| 30000|29200.000000005355|
+--------------------+------+------------------+



In [ ]:
pred.meanSquared_result.meanAbsoluteError, pred_result.meanSquared